In [28]:
def decorate(func):

    def wrap():
        print(f"{func.__name__} starting...")
        func()
        print(f"...stopping {func.__name__}")

    return wrap

There are two ways of decorating a function

In [29]:
@decorate
def func():
    print('Hello from inside func')

In [30]:
def hello():
    print("Hello World")

hello = decorate(hello)

In [31]:
func()

func starting...
Hello from inside func
...stopping func


In [32]:
hello()

hello starting...
Hello World
...stopping hello


In [33]:
@decorate
def add(a, b):
    return a + b

In [34]:
add(1,1)

TypeError: decorate.<locals>.wrap() takes 0 positional arguments but 2 were given

We got an error because the decorator is not equipped to handle a function which has arguments.

<wrong>A function AND its arguments are passed to the decorator as parameters.</wrong>

In [35]:
def decorate_well(func, *args, **kwargs):
    def wrapper(*args, **kwargs):
        print("Here we start....")
        func(*args, **kwargs)
        print("....Job well done!")
    return wrapper

In [36]:
@decorate_well
def add(a, b):
    return a + b

In [37]:
result = add(1,3)
print(result)

Here we start....
....Job well done!
None


The above decorator worked but add() did not return the expected result. It returned 'None' instead of the sum

In [38]:
def new_decorator(func, *args, **kwargs):
    def wrapper(*args, **kwargs):
        print("Beginning computation...")
        result = func(*args, **kwargs) # we calculate the result
        print("Ending computation...")
    return result

In [39]:
@new_decorator
def mmultiply(num1, num2):
    return num1 * num2


In [40]:
mmultiply(2,3)

TypeError: 'NoneType' object is not callable

Still does not work. Let us check what type is mmultiply

In [41]:
type(mmultiply)

NoneType

In [42]:
mult = multiply(2, 3)
print(mult)

NameError: name 'multiply' is not defined

The decorator still needs fixing. The inner function is not returning anything. The outer function is trying to access 'result' and return it. But 'result' stops being defined when the inner function exits without returning anything.

In [43]:
def og_decorator(func, *args, **kwargs):
    def wrapper(*args, **kwargs):
        print("Beginning computation...")
        result = func(*args, **kwargs)
        print("Ending computation...")
        return result # added this line to fix
    return wrapper # earlier we were returning result

In [44]:
@og_decorator
def div(a,b):
    return a/b

d = div(2,4)
print(d)

Beginning computation...
Ending computation...
0.5


I think we can lose the *args and **kwargs in inner function decorator. Let us check it

In [45]:
def test_decorator(func, *args, **kwargs):
    def wrapper(): # changed this line to remove the *args, **kwargs
        print("Beginning computation...")
        result = func(*args, **kwargs)
        print("Ending computation...")
        return result
    return wrapper

@test_decorator
def test_function(a, b):
    return a + b

print(test_function(1, 2))

TypeError: test_decorator.<locals>.wrapper() takes 0 positional arguments but 2 were given

It turns out that we cannot. Instead we can lose *args and **kwargs from the outer function definition of the decorator.

In [46]:
def test_decorator(func):  # Only takes the function as an argument
    def wrapper(*args, **kwargs):  # Wrapper needs *args and **kwargs
        print("Beginning computation...")
        result = func(*args, **kwargs)
        print("Ending computation...")
        return result
    return wrapper

@test_decorator
def test_function(a, b):
    return a + b

print(test_function(1, 2))

Beginning computation...
Ending computation...
3


when we decorate the function with @test_decorator, it is equivalent to

test_function = test_decorator(test_function)

so a function was provided and another function was returned. No requirement for *args and **kwargs.

The returned function (the inner function) must be able to handle the arguments that were passed to the original function. Hence its defintion should include *args and **kwargs

The outer function must take *args and **kwargs if we want to pass parameters to the decorator itself

In [48]:
def param_decorator(func, val=None):
    """val is the arguments passed to the decorator"""

    def inner(*args, **kwargs):
        """*args and **kwargs are the arguments passed to the decorated function"""
        print("hello from inside the decorator...")
        print(f"val is {val}")
        return func(*args, **kwargs)

@param_decorator(10)
def func(a, b):
    return a+b

func(1, 2)


TypeError: 'NoneType' object is not callable

In [50]:
def param_decorator(func, val=None):
    """val is the arguments passed to the decorator"""

    def inner(*args, **kwargs):
        """*args and **kwargs are the arguments passed to the decorated function"""
        print("hello from inside the decorator...")
        print(f"val is {val}")
        return func(*args, **kwargs)
    
    return inner    # this is a must for decorators

@param_decorator(10)
def func(a, b):
    return a+b

func(1, 2)

hello from inside the decorator...
val is None


TypeError: 'int' object is not callable

In [53]:
def param_decorator(func, val=None):
    """val is the arguments passed to the decorator"""

    def inner(*args, **kwargs):
        """*args and **kwargs are the arguments passed to the decorated function"""
        print("hello from inside the decorator...")
        print(f"val is {val}")
        return func(*args, **kwargs)
    
    return inner

@param_decorator(val=10) # keyword only argument
def func(a, b):
    return a+b

func(1, 2)

TypeError: param_decorator() missing 1 required positional argument: 'func'

In [55]:
def param_decorator(func, val): # made argument positional
    """val is the arguments passed to the decorator"""

    def inner(*args, **kwargs):
        """*args and **kwargs are the arguments passed to the decorated function"""
        print("hello from inside the decorator...")
        print(f"val is {val}")
        return func(*args, **kwargs)
    
    return inner

@param_decorator(10)
def func(a, b):
    return a+b

func(1, 2)

TypeError: param_decorator() missing 1 required positional argument: 'val'

In [56]:
def param_decorator(func, val): # made argument positional
    """val is the arguments passed to the decorator"""

    def inner(*args, **kwargs):
        """*args and **kwargs are the arguments passed to the decorated function"""
        print("hello from inside the decorator...")
        print(f"val is {val}")
        return func(*args, **kwargs)
    
    return inner

@param_decorator(10)
def func(a, b):
    return a+b

print(func(1, 2))

TypeError: param_decorator() missing 1 required positional argument: 'val'

It turns out the code above does not work because the outer function keeps expecting a function and a value. but the @ syntax only provides it with value=10.

To fix this we need to wrap the outer function in another function which takes only the value and returns a decorator which takes the function as the argument.

In [57]:
def this_returns_a_decorator(val):
    def the_decorator(func):
        def wrapper(*args, **kwargs):
            print(f"val is {val}")
            return func(*args, **kwargs)
        return wrapper
    return the_decorator

@this_returns_a_decorator(10)
def my_func(a, b):
    return a + b

print(my_func(1, 2))

val is 10
3
